# Hiệu suất của từng món ăn

## Dữ liệu cần thiết:
- Tên món ăn
- Danh mục món ăn
- Số lượng bán
- Doanh thu
- Ngày bán
## Các cột tính toán thêm:
- Doanh thu trung bình mỗi món
- Xếp hạng món ăn theo số lượng bán và doanh thu
- Tỷ lệ món ăn trong tổng doanh thu
- Xu hướng tiêu thụ theo ngày (ví dụ: món nào bán chạy nhất vào thứ Hai)
## Phương pháp:
Sử dụng Pivot Table và Pivot Chart để phân tích xu hướng theo thời gian và so sánh hiệu suất giữa các món ăn.
Sử dụng biểu đồ Pareto để xác định 20% món ăn tạo ra 80% doanh thu.

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import sys
import os
sys.path.append('../../../scripts/')
import data_cleaning

# 1. Import data

## 1.1 Import

In [20]:
df_menuitem = pd.read_csv("../../../data/processed/20240722/2.menuitem_20240722.csv")
df_ordersale = pd.read_csv("../../../data/processed/20240722/1.pos_ordersale_20240722.csv")
df_store = pd.read_csv("../../../data/processed/20240722/3.store_restaurant_20240722.csv")
df_menu_items = pd.read_csv("../../../data/processed/20240722/4.menu_items_20240722.csv")

In [21]:
df_menuitem.head()
df_ordersale.head()
df_store.head()
df_menu_items.head()

,MenuItemName,MenuItemDescription,PLU,MenuItemId,RecipeId
0,FtL/Ham,Ham FtLong,111000004,1,6
1,Six/Ham,Ham 6 inch,121000004,2,7
2,FfB/Ham,Ham FtFbd,112000004,3,2
3,fBd/Ham,Ham FlatBd,122000004,4,8
4,Sld/Ham,Ham Salad,131000004,5,9


## 1.2 Merge

In [22]:
df_ordersale.columns

Index(['MD5KEY_ORDERSALE', 'ChangeReceived', 'OrderNumber', 'TaxAmount',
       'MealLocation', 'TransactionId', 'StoreNumber', 'date'],
      dtype='object')

In [23]:
df_menuitem.columns

Index(['MD5KEY_MENUITEM', 'MD5KEY_ORDERSALE', 'CategoryDescription',
       'DepartmentDescription', 'Description', 'StoreNumber', 'TaxAmount',
       'AdjustedPrice', 'DiscountAmount', 'Price', 'Quantity', 'PLU', 'Id',
       'date', 'SalesAmount', 'day_of_week'],
      dtype='object')

In [24]:
df_store.columns

Index(['STORE_ADDRESS1', 'DISTRIBUTION_REGION', 'STORE_STATE', 'STORE_CITY',
       'STORE_ZIP', 'STORE_TYPE', 'STORE_LOYALTY_FLAG', 'STORE_NUMBER'],
      dtype='object')

In [25]:
df_menu_items.columns

Index(['MenuItemName', 'MenuItemDescription', 'PLU', 'MenuItemId', 'RecipeId'], dtype='object')

In [26]:
df = pd.merge(df_ordersale, df_menuitem, on="MD5KEY_ORDERSALE", how="left")


In [29]:
df.columns

Index(['MD5KEY_ORDERSALE', 'ChangeReceived', 'OrderNumber', 'TaxAmount_x',
       'MealLocation', 'TransactionId', 'StoreNumber_x', 'date_x',
       'MD5KEY_MENUITEM', 'CategoryDescription', 'DepartmentDescription',
       'Description', 'StoreNumber_y', 'TaxAmount_y', 'AdjustedPrice',
       'DiscountAmount', 'Price', 'Quantity', 'PLU', 'Id', 'date_y',
       'SalesAmount', 'day_of_week'],
      dtype='object')

In [34]:
df = pd.merge(df, df_store, left_on="StoreNumber_x", right_on="STORE_NUMBER", how="left")

In [35]:
df.columns

Index(['MD5KEY_ORDERSALE', 'ChangeReceived', 'OrderNumber', 'TaxAmount_x',
       'MealLocation', 'TransactionId', 'StoreNumber_x', 'date_x',
       'MD5KEY_MENUITEM', 'CategoryDescription', 'DepartmentDescription',
       'Description', 'StoreNumber_y', 'TaxAmount_y', 'AdjustedPrice',
       'DiscountAmount', 'Price', 'Quantity', 'PLU', 'Id', 'date_y',
       'SalesAmount', 'day_of_week', 'STORE_ADDRESS1', 'DISTRIBUTION_REGION',
       'STORE_STATE', 'STORE_CITY', 'STORE_ZIP', 'STORE_TYPE',
       'STORE_LOYALTY_FLAG', 'STORE_NUMBER'],
      dtype='object')

In [37]:
df = pd.merge(df, df_menu_items, left_on="Id", right_on="MenuItemId", how="left")

In [39]:
print(df['SalesAmount'].sum())
print(df_menuitem['SalesAmount'].sum())

422607.57
422607.57


In [43]:
df.columns

Index(['MD5KEY_ORDERSALE', 'ChangeReceived', 'OrderNumber', 'TaxAmount_x',
       'MealLocation', 'TransactionId', 'StoreNumber_x', 'date_x',
       'MD5KEY_MENUITEM', 'CategoryDescription', 'DepartmentDescription',
       'Description', 'StoreNumber_y', 'TaxAmount_y', 'AdjustedPrice',
       'DiscountAmount', 'Price', 'Quantity', 'PLU_x', 'Id', 'date_y',
       'SalesAmount', 'day_of_week', 'STORE_ADDRESS1', 'DISTRIBUTION_REGION',
       'STORE_STATE', 'STORE_CITY', 'STORE_ZIP', 'STORE_TYPE',
       'STORE_LOYALTY_FLAG', 'STORE_NUMBER', 'MenuItemName',
       'MenuItemDescription', 'PLU_y', 'MenuItemId', 'RecipeId'],
      dtype='object')

# 2. Thêm các cột
- QuantityRank: Xếp hạng số lượng bán
- RevenueRank: Xếp hạng doanh thu
- RevenuePercentage: tỷ lệ món ăn trong tổng doanh thu

In [47]:
item_performance = df.groupby('MenuItemName').agg(
  total_quantity=('Quantity', 'sum'),
  SalesAmount=('SalesAmount', 'sum')
)

item_performance

,total_quantity,SalesAmount
MenuItemName,,
#NAME?,2075.0,2610.95
21oz21CDrk,10586.0,16932.57
30oz30CDrk,1133.0,2133.05
40oz40CDrk,612.0,1276.78
6Bf/BcnEgg,47.0,192.44
...,...,...
fBd/TrkyHm,2.0,9.51
fBd/Tuna,30.0,126.91
fBd/Turkey,70.0,329.21
